# 학습된 Checkpoint를 저장하고 불러온 후, 활용

 - graph를 freeze 한 후, 이용하기 위한 방법을 기술
 - 그래프를 freeze 할 때, graph_def ProtoBuf 형식으로 저장하였음
 - Python으로 로드하기 위해서는, graph_def ProtoBuf를 Import 하고, graph_def를 Graph로 로드해야함


In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework.graph_util import convert_variables_to_constants

/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_graph(frozen_graph_filename):
    
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        #protobuf file을 불러오고, parse 한다
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='prefix')
        
    return graph

In [3]:
graph = load_graph('./results/frozen_model.pb')

In [4]:
#각각 opearation 불러오기
for op in graph.get_operations():
    print(op.name)

prefix/Placeholder/inputs_placeholder
prefix/NN/W1
prefix/NN/W1/read
prefix/NN/b1
prefix/NN/b1/read
prefix/NN/W2
prefix/NN/W2/read
prefix/NN/b2
prefix/NN/b2/read
prefix/NN/MatMul
prefix/NN/add
prefix/NN/Relu
prefix/NN/MatMul_1
prefix/NN/add_1
prefix/NN/Relu_1
prefix/NN/Add
prefix/NN/truediv/y
prefix/NN/truediv
prefix/Accuracy/predictions/y
prefix/Accuracy/predictions


In [5]:
# input과 output nodes 접근
x = graph.get_tensor_by_name('prefix/Placeholder/inputs_placeholder:0')
y = graph.get_tensor_by_name('prefix/Accuracy/predictions:0')

In [9]:
with tf.Session(graph=graph) as sess:
    # Note: we don't nee to initialize/restore anything
    # 그래프에 Variables이 없고, only 하드코딩 숫자들만 존재
    y_out = sess.run(y, feed_dict={
        x: [[3, 5, 7, 4, 5, 1, 1, 1, 1, 1]] # < 45
    })
    # 이 뉴럴넷은 45보다 크기가 클 경우에, 인식하게 만들었다.
    # 그 외는 False
    print(y_out) # [[ False ]] Yay, it works!

[[False]]


# Appendix

https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc

https://blog.metaflow.fr/tensorflow-saving-restoring-and-mixing-multiple-models-c4c94d5d7125

http://cv-tricks.com/how-to/freeze-tensorflow-models/